## Часть 1: Парсинг

In [3]:
# Ячейка с импортами
import requests as rq
from bs4 import BeautifulSoup

import pandas as pd

from time import sleep

В качестве материала для парсинга я выбрала книжный раздел на сайте Мир Фантастики: https://www.mirf.ru/category/book/. Там есть несколько видов статей: рецензии на книги, отрывки из произведений, короткие рассказы, мастер-классы.

### 1. Парсинг ссылок на книжные страницы

In [ ]:
review_data = []

In [1]:
# Функция для парсинга url со страниц со статьями. Верхние статьи на каждой странице лежат в отдельных тегах.
def review_parse(url):
    
    page = rq.get(url)
    soup = BeautifulSoup(page.content, 'html.parser')
    # Для первой книги на странице идет свой тег - отдельно найдем описание и url
    first_book_descr = soup.find("div", class_ = "home_article_first_left_item_desc_text").text.replace('\n', '')
    first_book_url = soup.find("div", class_ = "home_article_first_left_item_desc_text").find('a').get('href')
    review_data.append([first_book_descr, first_book_url])

    # Здесь парсинг следующих топовых рецензий - они в отдельном подклассе
    next_reviews = soup.find('div', class_ = 'home_article_first_right').findAll('div', class_ = 'home_article_item_desc_text')
    for item in next_reviews:
        book_title = item.text.strip()
        book_url = item.find('a').get('href')
        review_data.append([book_title, book_url])

    # остаток рецензий
    rest_of_reviews = soup.find('div', {'id': 'home_articles'}).findAll('div', class_ = "home_article_item_desc_text")
    for review in rest_of_reviews:
        review_title = review.text.strip()
        review_url = review.find('a').get('href')
        review_data.append([review_title, review_url])

In [ ]:
# Всего на сайте 81 страница со статьями
for page_number in range(1, 82):
    print(page_number)
    source_url = f"https://www.mirf.ru/category/book/page/{page_number}"
    review_parse (source_url)
    sleep(5)

In [ ]:
# Для удобства дальнейшего использования ссылки положила в датафрейм.
the_review_df = pd.DataFrame(review_data, columns = ['name', 'url'])

### 2. Парсинг информации с книжных страниц

In [ ]:
book_leads = []
book_highlights = []
book_results = []

In [ ]:
# На страницах статей есть 3 типа разделов, в которых хранится главная информация о книге: lead, highlight и итог. 
# В дальнейшем я буду использовать только тексты, хранящиеся в лиде, но на этом этапе на всякий случай собрала их все, 
# т.к. не была уверена в их итоговом количестве.
def GetBookInfo(url):
    bookie = rq.get(url)
    next_soup = BeautifulSoup(bookie.content, 'html.parser')
    
    try:
        book_lead = next_soup.find('div', class_ = "lead").text
    except:
        book_lead = "Absent"

    try:
        book_highlight = next_soup.find('div', class_ = "highlight").text
    except:
        book_highlight = "Absent"

    try:
        book_result = next_soup.findAll('div', class_ = "vrezka2")[1].text
    except:
        book_result = 'Absent'

    book_leads.append(book_lead)
    book_highlights.append(book_highlight)
    book_results.append(book_result)

In [ ]:
for j in range(len(final_review_df)):
    new_url = final_review_df.url[j]
    print(j)
    GetBookInfo(new_url)
    sleep(5)

In [ ]:
the_review_df['leads'] = book_leads
the_review_df['highlights'] = book_highlights
the_review_df['results'] = book_results

In [ ]:
# Чтобы не парсить 1500+ страниц статей каждый раз, результаты парсинга я положила в csv файл, который и будет использоваться ниже 

the_review_df.to_csv('25_10_final_magolego_reviews.csv')

## Часть 2: препроцессинг

In [4]:
the_review_df = pd.read_csv(r'25_10_final_magolego_reviews.csv')
the_review_df.drop('Unnamed: 0', axis = 1, inplace = True)

In [6]:
the_review_df

,name,url,leads,highlights,results
0,Лучшие книги про героев-дипломатов и мирные ре...,https://www.mirf.ru/book/luchshie-knigi-pro-ge...,Absent,\nФэнтези и фантастика ассоциируются с увлекат...,Absent
1,Тэд Уильямс «Братья ветра». Приквел знаменитог...,https://www.mirf.ru/book/tad-williams-bratya-v...,\nЗа тысячу лет до начала событий первой трило...,Absent,"Итог: крепкий роман о битве с драконом, оборач..."
2,"«А вы, батенька, случайно не Мэри Сью?» — отры...",https://www.mirf.ru/book/a-vy-batenka-sluchajn...,\nУ нас на сайте — отрывок из руководства по с...,Absent,Absent
3,Дэвид Гемелл «Йон Шэнноу»: постапокалиптически...,https://www.mirf.ru/book/devid-gemell-jon-shen...,Absent,\nТрилогия Дэвида Геммела про Йона Шэнноу не п...,Absent
4,Эдуард Веркин «снарк снарк». Русский «стивенки...,https://www.mirf.ru/book/eduard-verkin-snark-s...,\nТридцатилетний писатель Виктор возвращается ...,Absent,"Итог: роман-эпопея с гоголевскими типажами, оч..."
...,...,...,...,...,...
1597,Классики. Филип Дик,https://www.mirf.ru/book/fantasty/klassiki-phi...,"Автор 30 с лишним романов и 110 рассказов, Фил...",Absent,Absent
1598,Юрий Нестеренко «Как снять культовый фильм» (п...,https://www.mirf.ru/fun/funny/yuriy-nesterenko...,Absent,Иллюстрации: Sonymax Studios (www.sonymax.ru),Absent
1599,«Хроники Амбера» — миры и отражения Роджера Же...,https://www.mirf.ru/book/hroniki-ambera-miry-i...,Absent,Absent,Absent
1600,"Продолжения Толкина. Ник Перумов, Ниэнна и другие",https://www.mirf.ru/book/prodolzheniya-tolkina...,Absent,"Прошло уже более полувека после того, как книг...",Absent


In [14]:
# В дальнейшем используется датасет, состоящий исключительно из лидов - 1115 текстов
only_leads = the_review_df[the_review_df.leads != 'Absent']
only_leads = only_leads[["name", "leads"]]
only_leads.reset_index(drop= True , inplace= True )

In [8]:
len(only_leads)

1115

In [9]:
only_leads

,name,leads
0,Тэд Уильямс «Братья ветра». Приквел знаменитог...,\nЗа тысячу лет до начала событий первой трило...
1,"«А вы, батенька, случайно не Мэри Сью?» — отры...",\nУ нас на сайте — отрывок из руководства по с...
2,Эдуард Веркин «снарк снарк». Русский «стивенки...,\nТридцатилетний писатель Виктор возвращается ...
3,Танец под звёздами. Мини-история из «Ключи Лок...,\nУ нас на сайте — отрывок из мистического ком...
4,Мария Вой «Сиротки». Мрачные приключения шлюхи,\nПоследний колдун королевства Свортек проводи...
...,...,...
1110,Сюзанна Кларк «Джонатан Стрендж и мистер Норрелл»,"Говорят, что фэнтези себя исчерпало и не может..."
1111,Контакт: Джордж Мартин,\nСреди всех современных авторов фэнтези нет т...
1112,Эротическая фантастика,"Эффектная длинноногая блондинка, паря в невесо..."
1113,Классики. Филип Дик,"Автор 30 с лишним романов и 110 рассказов, Фил..."


In [16]:
# Текст до препроцессинга:
only_leads.leads[0]

'\nЗа тысячу лет до начала событий первой трилогии Тэда Уильямса «Память, Скорбь и Шип» в Светлом Арде царствуют ситхи, а люди даже не думают бросать им вызов, преклоняясь перед их силой и мудростью. Однажды в землях человеческого королевства появляется древний дракон, и люди просят повелителей города Асу’а помочь победить чудовище. Молодой принц ситхи Инелуки даёт клятву убить дракона, но это решение приводит к ужасающим последствиям для самого Инелуки и его старшего брата Хакатри, а также для всей расы ситхи. Несколько неосторожных слов навсегда изменили историю Светлого Арда, и их отголоски по-прежнему дают о себе знать новым поколениям.\n'

### Избавимся от лишних символов

In [11]:
# латинские буквы, цифры, знаки пунктуации, дополнительные символы, лишние пробелы
english = (r'[a-z]+')
digits = (r'\d+')
punctuation = (r'[^\w\s]')
extra = (r'\n?')
symbols = (r'[\n\xa0]')
extra_spaces = (r'\s{2,}')

In [17]:
only_leads.leads = only_leads.leads.str.replace(english, ' ')
only_leads.leads = only_leads.leads.str.replace(digits, ' ')
only_leads.leads = only_leads.leads.str.replace(punctuation, ' ')
only_leads.leads = only_leads.leads.str.replace(extra, '')
only_leads.leads = only_leads.leads.str.replace(symbols, ' ')
only_leads.leads = only_leads.leads.str.replace(extra_spaces, ' ')

C:\Users\marus\AppData\Local\Temp/ipykernel_18972/1454801353.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  only_leads.leads = only_leads.leads.str.replace(english, ' ')
C:\Users\marus\AppData\Local\Temp/ipykernel_18972/1454801353.py:2: FutureWarning: The default value of regex will change from True to False in a future version.
  only_leads.leads = only_leads.leads.str.replace(digits, ' ')
C:\Users\marus\AppData\Local\Temp/ipykernel_18972/1454801353.py:3: FutureWarning: The default value of regex will change from True to False in a future version.
  only_leads.leads = only_leads.leads.str.replace(punctuation, ' ')
C:\Users\marus\AppData\Local\Temp/ipykernel_18972/1454801353.py:4: FutureWarning: The default value of regex will change from True to False in a future version.
  only_leads.leads = only_leads.leads.str.replace(extra, '')
C:\Users\marus\AppData\Local\Temp/ipykernel_18972/1454801353.py:5: FutureWarning: The default value 

In [18]:
# Текст после очистки от лишних символов
only_leads.leads[0]

'За тысячу лет до начала событий первой трилогии Тэда Уильямса Память Скорбь и Шип в Светлом Арде царствуют ситхи а люди даже не думают бросать им вызов преклоняясь перед их силой и мудростью Однажды в землях человеческого королевства появляется древний дракон и люди просят повелителей города Асу а помочь победить чудовище Молодой принц ситхи Инелуки даёт клятву убить дракона но это решение приводит к ужасающим последствиям для самого Инелуки и его старшего брата Хакатри а также для всей расы ситхи Несколько неосторожных слов навсегда изменили историю Светлого Арда и их отголоски по прежнему дают о себе знать новым поколениям '

### Токенизируем тексты

In [21]:
import nltk

In [24]:
from nltk.tokenize import word_tokenize

In [25]:
tokenized_leads = []
for sentence in only_leads.leads:
    sentence = word_tokenize(sentence)
    tokenized_leads.append(sentence)

In [30]:
tokenized_leads[0][0:20]

['За',
 'тысячу',
 'лет',
 'до',
 'начала',
 'событий',
 'первой',
 'трилогии',
 'Тэда',
 'Уильямса',
 'Память',
 'Скорбь',
 'и',
 'Шип',
 'в',
 'Светлом',
 'Арде',
 'царствуют',
 'ситхи',
 'а']

### Лемматизация и удаление стоп-слов

In [28]:
from nltk.corpus import stopwords
nltk.download('stopwords')
russian_stopwords = stopwords.words('russian')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\marus\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [63]:
# Здесь дополнительно добавила слова, которые встречались слишком часто и во всех выделенных топиках.
# Слова "мир" и "фантастика" добавлены из-за того, что они есть в названии журнала (и сайта),
# и поэтому слишком часто используются в текстах ("мирф" - сокращенное название)
extra_stopwords = [
    'дом',
    'который',
    'год',
    'это',
    'весь',
    'всё',
    'книга',
    'мир',
    'фантастика',
    'мирф',
    'роман',
    'рассказ',
    'свой',
    'его',
    'её'
]

In [27]:
!pip3 install pymorphy2

In [31]:
# Для лемматизации я выбрала pymorphy2, т.к. он хорошо работает с русскими текстами и обычно не ошибается с леммами.
import pymorphy2

In [32]:
morph = pymorphy2.MorphAnalyzer()

In [64]:
clean_lemmas_leads = []
for document in tokenized_leads:
    temp = []
    for word in document:
        word = word.lower()
        lemma = morph.parse(word)[0].normal_form
        if lemma not in russian_stopwords and lemma not in extra_stopwords:
            temp.append(lemma)
    clean_lemmas_leads.append(temp)

In [65]:
clean_lemmas_leads[0][0:20]

['тысяча',
 'начало',
 'событие',
 'первый',
 'трилогия',
 'тэда',
 'уильямс',
 'память',
 'скорбь',
 'шип',
 'светлый',
 'ард',
 'царствовать',
 'ситх',
 'человек',
 'думать',
 'бросать',
 'вызов',
 'преклоняться',
 'сила']

# Часть 3: тематическое моделирование

In [37]:
from gensim import corpora
from gensim import models

In [95]:
# Coherence model для проверки
from gensim.models import CoherenceModel

In [66]:
dictionary = corpora.Dictionary(clean_lemmas_leads)
corpus = [dictionary.doc2bow(text) for text in clean_lemmas_leads]

In [67]:
tfidf_model = models.TfidfModel(corpus)
tfidf = tfidf_model[corpus]

### LSI

Изначально и для LSI, и для LDA выбрала число топиков = 5

In [68]:
lsi = models.LsiModel(tfidf, id2word=dictionary, num_topics=5)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(


In [70]:
lsi.show_topics(num_words=10)

[(0,
  '0.123*"автор" + 0.122*"новый" + 0.121*"издательство" + 0.110*"отрывок" + 0.109*"фэнтези" + 0.107*"наш" + 0.106*"самый" + 0.104*"человек" + 0.104*"первый" + 0.101*"история"'),
 (1,
  '0.411*"миниатюра" + 0.403*"пётр" + 0.403*"бормор" + 0.303*"впервые" + 0.246*"опубликовать" + 0.220*"эпиграф" + 0.187*"утро" + 0.157*"утренний" + 0.094*"снова" + 0.091*"воскресный"'),
 (2,
  '-0.201*"книжный" + -0.163*"рекомендация" + -0.161*"наш" + -0.157*"итог" + -0.150*"новинка" + 0.146*"человек" + -0.129*"автор" + -0.122*"редактор" + -0.120*"издатель" + -0.117*"топ"'),
 (3,
  '0.305*"отрывок" + 0.244*"издательство" + 0.242*"сайт" + 0.160*"разрешение" + 0.128*"публиковать" + 0.127*"фрагмент" + 0.122*"сборник" + 0.122*"трилогия" + -0.116*"итог" + -0.115*"рекомендация"'),
 (4,
  '-0.214*"рекомендация" + -0.206*"итог" + -0.186*"отрывок" + 0.181*"новинка" + -0.146*"блогер" + -0.146*"рекомендовать" + -0.146*"голосовать" + -0.146*"подводить" + -0.143*"сайт" + -0.140*"публиковать"')]

- Интересен топик 1: Пётр Бормор - автор "микрорассказов", которые публикуются на сайте практически ежемесячно, так что их выделение в отдельный топик закономерно (тем более со словами "миниатюра", "впервые", "опубликовать"). 
- Топик 3 содержит слова "отрывок", "издательство", "разрешение", "фрагмент" - скорее всего, к нему относятся отрывки из книг, опубликованные на сайте.
- В топике 4 выделяются слова "рекомендация", "рекомендовать", "блогер", так что можно предположить, что к нему отнесли статьи с рекомендациями от авторов сайта и блогеров, однако к нему же относятся слова "голосовать" и "подводить" (видимо, статьи с конкурсами или голосованиями за книгу), хотя их хорошо было бы выделить в отдельный топик.
- Топики 0 и 2 частично дублируют остальные топики, по ним сложно понять, к каким именно статьям они относятся.

In [113]:
coh_model = CoherenceModel(model = lsi,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_lsi_5 = coh_model.get_coherence()
print(coh_score_lsi_5)

0.479451080979498


Попробуем уменьшить число топиков

In [114]:
lsi_4 = models.LsiModel(tfidf, id2word=dictionary, num_topics=4)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(


In [115]:
lsi_4.show_topics(num_words=10)

[(0,
  '-0.123*"автор" + -0.121*"новый" + -0.121*"издательство" + -0.111*"отрывок" + -0.109*"фэнтези" + -0.108*"наш" + -0.106*"самый" + -0.104*"первый" + -0.103*"человек" + -0.102*"история"'),
 (1,
  '0.410*"миниатюра" + 0.401*"бормор" + 0.401*"пётр" + 0.304*"впервые" + 0.245*"опубликовать" + 0.220*"эпиграф" + 0.186*"утро" + 0.166*"утренний" + 0.097*"воскресный" + 0.093*"снова"'),
 (2,
  '0.200*"книжный" + 0.162*"рекомендация" + 0.159*"наш" + 0.154*"итог" + 0.147*"новинка" + -0.139*"человек" + 0.128*"автор" + 0.122*"редактор" + 0.115*"топ" + 0.112*"издатель"'),
 (3,
  '-0.303*"отрывок" + -0.241*"сайт" + -0.240*"издательство" + -0.155*"разрешение" + -0.128*"сборник" + -0.125*"трилогия" + 0.125*"рекомендация" + 0.123*"итог" + -0.122*"фрагмент" + -0.118*"публиковать"')]

In [119]:
coh_model = CoherenceModel(model = lsi_4,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_lsi_4 = coh_model.get_coherence()
print(coh_score_lsi_4)

0.7780125205697657


In [116]:
lsi_3 = models.LsiModel(tfidf, id2word=dictionary, num_topics=3)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\gensim\models\lsimodel.py:963: DeprecationWarning: Please use `csc_matvecs` from the `scipy.sparse` namespace, the `scipy.sparse.sparsetools` namespace is deprecated.
  sparsetools.csc_matvecs(


In [117]:
lsi_3.show_topics(num_words=10)

[(0,
  '0.122*"автор" + 0.122*"новый" + 0.121*"издательство" + 0.111*"отрывок" + 0.109*"фэнтези" + 0.108*"наш" + 0.106*"самый" + 0.104*"человек" + 0.104*"первый" + 0.102*"история"'),
 (1,
  '-0.409*"миниатюра" + -0.401*"бормор" + -0.401*"пётр" + -0.306*"впервые" + -0.244*"опубликовать" + -0.220*"эпиграф" + -0.185*"утро" + -0.164*"утренний" + -0.095*"снова" + -0.093*"воскресный"'),
 (2,
  '-0.196*"книжный" + -0.165*"рекомендация" + -0.164*"наш" + -0.159*"итог" + -0.148*"новинка" + 0.137*"человек" + -0.124*"редактор" + -0.122*"автор" + -0.122*"издатель" + -0.119*"топ"')]

In [120]:
coh_model = CoherenceModel(model = lsi_3,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_lsi_3 = coh_model.get_coherence()
print(coh_score_lsi_3)

0.3640059069971679


Для модели LSI наивысший coherense score при числе топиков = 4.

### LDA

In [71]:
lda = models.LdaModel(tfidf, id2word=dictionary, num_topics=5)

In [75]:
for idx, topic in lda.print_topics():
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"издательство" + 0.001*"миниатюра" + 0.001*"пётр" + 0.001*"бормор" + 0.001*"сайт" + 0.001*"новый" + 0.001*"публиковать" + 0.001*"фэнтези" + 0.001*"отрывок" + 0.001*"самый"
Topic: 1 Word: 0.001*"человек" + 0.001*"отрывок" + 0.001*"мочь" + 0.001*"самый" + 0.001*"время" + 0.001*"автор" + 0.001*"стать" + 0.001*"жизнь" + 0.001*"война" + 0.001*"новый"
Topic: 2 Word: 0.001*"новый" + 0.001*"история" + 0.001*"автор" + 0.001*"первый" + 0.001*"человек" + 0.001*"фантастический" + 0.001*"самый" + 0.001*"несколько" + 0.001*"стать" + 0.001*"отрывок"
Topic: 3 Word: 0.001*"новый" + 0.001*"стать" + 0.001*"автор" + 0.001*"время" + 0.001*"история" + 0.001*"издательство" + 0.001*"ещё" + 0.001*"война" + 0.001*"наш" + 0.001*"выйти"
Topic: 4 Word: 0.001*"человек" + 0.001*"сайт" + 0.001*"отрывок" + 0.001*"земля" + 0.001*"первый" + 0.001*"хороший" + 0.001*"новый" + 0.001*"история" + 0.001*"друг" + 0.001*"ещё"


In [54]:
# Визуализация
!pip install pyLDAvis

In [72]:
import pyLDAvis.gensim_models
pyLDAvis.enable_notebook()

In [73]:
vis = pyLDAvis.gensim_models.prepare(lda, corpus, dictionary=lda.id2word)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [74]:
vis

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
1      0.016780 -0.005557       1        1  21.412453
3     -0.017533 -0.018819       2        1  20.647530
0     -0.012368  0.020607       3        1  19.731237
2      0.003034  0.002818       4        1  19.251472
4      0.010087  0.000952       5        1  18.957307, topic_info=           Term       Freq      Total Category  logprob  loglift
1336       утро  10.000000  10.000000  Default  30.0000  30.0000
2618       рина   8.000000   8.000000  Default  29.0000  29.0000
3367  миниатюра  17.000000  17.000000  Default  28.0000  28.0000
4027       пётр  16.000000  16.000000  Default  27.0000  27.0000
4025     бормор  16.000000  16.000000  Default  26.0000  26.0000
...         ...        ...        ...      ...      ...      ...
154    странный   4.097943  17.662593   Topic5  -7.6023   0.2020
5         город   4.272694  21.199117   Topic5  -7.5606   0.0613
283       жизнь   4.345307  25.047515   Topic5  -7.5437  -0.0887
585       война   4.299808  25.984861   Topic5  -7.5542  -0.1360
1019        наш   4.179755  26.221898   Topic5  -7.5826  -0.1734

[406 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
3042      1  0.089168        d
3042      2  0.089168        d
3042      3  0.356673        d
3042      4  0.267505        d
3042      5  0.267505        d
...     ...       ...      ...
3405      5  0.189445  эпиграф
5865      1  0.487103     юнис
5865      2  0.243551     юнис
5865      3  0.243551     юнис
5865      4  0.243551     юнис

[1457 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[2, 4, 1, 3, 5])

Из визуализации видно, что 1, 4 и 5 топики пересекаются. Проверим coherense:

In [125]:
coh_model = CoherenceModel(model = lda,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_lda_5 = coh_model.get_coherence()
print(coh_score_lda_5)

0.2615349057302908


Результат с 5 топиками равен 0.2615349057302908. Проверим, как изменится coherence при уменьшении числа топиков до 3-х:

In [82]:
lda_3_topics = models.LdaModel(tfidf, id2word=dictionary, num_topics=3)

In [87]:
for idx, topic in lda_3_topics.print_topics():
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"человек" + 0.001*"отрывок" + 0.001*"история" + 0.001*"время" + 0.001*"новый" + 0.001*"стать" + 0.001*"война" + 0.001*"самый" + 0.001*"мочь" + 0.001*"сайт"
Topic: 1 Word: 0.001*"хороший" + 0.001*"человек" + 0.001*"новый" + 0.001*"ещё" + 0.001*"издательство" + 0.001*"автор" + 0.001*"время" + 0.001*"часть" + 0.001*"самый" + 0.001*"первый"
Topic: 2 Word: 0.001*"автор" + 0.001*"новый" + 0.001*"отрывок" + 0.001*"самый" + 0.001*"история" + 0.001*"фэнтези" + 0.001*"фантастический" + 0.001*"сайт" + 0.001*"наш" + 0.001*"первый"


In [84]:
vis_3 = pyLDAvis.gensim_models.prepare(lda_3_topics, corpus, dictionary=lda.id2word)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [85]:
vis_3

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
0      0.000396 -0.012287       1        1  36.835367
2     -0.011202  0.005812       2        1  36.108295
1      0.010806  0.006475       3        1  27.056338, topic_info=             Term       Freq      Total Category  logprob  loglift
162       хороший  23.000000  23.000000  Default  30.0000  30.0000
3042            d  10.000000  10.000000  Default  29.0000  29.0000
365       конкурс   8.000000   8.000000  Default  28.0000  28.0000
2851  воскресение   7.000000   7.000000  Default  27.0000  27.0000
2260      книжный  16.000000  16.000000  Default  26.0000  26.0000
...           ...        ...        ...      ...      ...      ...
431         стать   6.383138  27.489557   Topic3  -7.5149  -0.1529
283         жизнь   6.046862  25.127941   Topic3  -7.5690  -0.1172
1019          наш   6.100276  26.464392   Topic3  -7.5602  -0.1602
94           сайт   5.978716  28.157564   Topic3  -7.5803  -0.2424
81        отрывок   6.058100  31.560823   Topic3  -7.5671  -0.3433

[262 rows x 6 columns], token_table=      Topic      Freq         Term
term                              
1255      1  0.518456            a
1255      2  0.103691            a
1255      3  0.414765            a
3042      1  0.185152            d
3042      2  0.277728            d
...     ...       ...          ...
2288      2  0.177756  эксперимент
2288      3  0.177756  эксперимент
3405      1  0.309512      эпиграф
3405      2  0.495220      эпиграф
3405      3  0.247610      эпиграф

[585 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[1, 3, 2])

Теперь топики не пересекаются, но по самым частным словам сложно понять, какие именно статьи попали в тот или иной топик

In [126]:
coh_model = CoherenceModel(model = lda_3_topics,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_lda_3 = coh_model.get_coherence()
print(coh_score_lda_3)

0.3206250828776468


При уменьшении числа топиков до 3-х coherence score увеличился на ~ 0.06, но все еще ниже, чем для модели lsi. Проверим результат для 4-х топиков:

In [127]:
lda_4_topics = models.LdaModel(tfidf, id2word=dictionary, num_topics=4)

In [128]:
coh_model = CoherenceModel(model = lda_4_topics,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_lda_4 = coh_model.get_coherence()
print(coh_score_lda_4)

0.31771910654568575


Coherence score при 4-х топиках незначительно ниже, чем при 3-х.

## LDA Multicore

In [105]:
lda_multicore = models.LdaMulticore(tfidf, num_topics=5, id2word=dictionary, passes=5)

In [124]:
for idx, topic in lda_multicore.print_topics():
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.001*"человек" + 0.001*"новый" + 0.001*"издательство" + 0.001*"автор" + 0.001*"первый" + 0.001*"время" + 0.001*"фантастический" + 0.001*"отрывок" + 0.001*"ещё" + 0.001*"публиковать"
Topic: 1 Word: 0.001*"человек" + 0.001*"время" + 0.001*"автор" + 0.001*"отрывок" + 0.001*"мочь" + 0.001*"история" + 0.001*"книжный" + 0.001*"наш" + 0.001*"самый" + 0.001*"сайт"
Topic: 2 Word: 0.001*"человек" + 0.001*"герой" + 0.001*"новый" + 0.001*"война" + 0.001*"самый" + 0.001*"жизнь" + 0.001*"мочь" + 0.001*"ещё" + 0.001*"однако" + 0.001*"стать"
Topic: 3 Word: 0.001*"история" + 0.001*"самый" + 0.001*"новый" + 0.001*"рассказывать" + 0.001*"автор" + 0.001*"первый" + 0.001*"наш" + 0.001*"несколько" + 0.001*"человек" + 0.001*"цикл"
Topic: 4 Word: 0.001*"фэнтези" + 0.001*"новый" + 0.001*"отрывок" + 0.001*"человек" + 0.001*"война" + 0.001*"фантастический" + 0.001*"самый" + 0.001*"автор" + 0.001*"стать" + 0.001*"город"


In [121]:
vis_multicore = pyLDAvis.gensim_models.prepare(lda_multicore, corpus, dictionary=lda.id2word)

C:\Users\marus\AppData\Local\Programs\Python\Python310\lib\site-packages\pyLDAvis\_prepare.py:247: FutureWarning: In a future version of pandas all arguments of DataFrame.drop except for the argument 'labels' will be keyword-only.
  by='saliency', ascending=False).head(R).drop('saliency', 1)


In [122]:
vis_multicore

PreparedData(topic_coordinates=              x         y  topics  cluster       Freq
topic                                                
2     -0.022433  0.009926       1        1  21.060247
1      0.018301  0.015436       2        1  20.871563
0      0.000215 -0.022497       3        1  20.624057
3      0.009098 -0.004891       4        1  20.583091
4     -0.005180  0.002026       5        1  16.861042, topic_info=              Term      Freq      Total Category  logprob  loglift
9370        нексус  5.000000   5.000000  Default  30.0000  30.0000
3834        говард  5.000000   5.000000  Default  29.0000  29.0000
8327         пьеса  6.000000   6.000000  Default  28.0000  28.0000
4664  рекомендация  7.000000   7.000000  Default  27.0000  27.0000
2715       вторник  7.000000   7.000000  Default  26.0000  26.0000
...            ...       ...        ...      ...      ...      ...
67             ещё  3.861092  27.501745   Topic5  -7.5447  -0.1831
20          начало  3.654857  19.961249   Topic5  -7.5996   0.0824
745        впервые  3.619302  17.865338   Topic5  -7.6093   0.1836
162        хороший  3.654098  24.282793   Topic5  -7.5998  -0.1138
27          первый  3.666664  28.104577   Topic5  -7.5963  -0.2565

[394 rows x 6 columns], token_table=      Topic      Freq     Term
term                          
273       1  0.111760        b
273       2  0.111760        b
273       3  0.447041        b
273       4  0.111760        b
273       5  0.111760        b
...     ...       ...      ...
440       5  0.137269     юный
8350      1  0.236741  ярмарка
8350      2  0.236741  ярмарка
8350      3  0.236741  ярмарка
8350      4  0.473483  ярмарка

[1330 rows x 3 columns], R=30, lambda_step=0.01, plot_opts={'xlab': 'PC1', 'ylab': 'PC2'}, topic_order=[3, 2, 1, 4, 5])

На данной визуализации видно, что для модели LDA Multicore топики не пересекаются. Проверим coherence:

In [129]:
coh_model = CoherenceModel(model = lda_multicore,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_multicore_5 = coh_model.get_coherence()
print(coh_score_multicore_5)

0.27289160560854947


Попробуем эту же модель с 4 и 3 топиками:

In [130]:
lda_multicore_4 = models.LdaMulticore(tfidf, num_topics=4, id2word=dictionary, passes=5)

In [131]:
coh_model = CoherenceModel(model = lda_multicore_4,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_multicore_4 = coh_model.get_coherence()
print(coh_score_multicore_4)

0.29049085003826214


In [132]:
lda_multicore_3 = models.LdaMulticore(tfidf, num_topics=3, id2word=dictionary, passes=5)

In [133]:
coh_model = CoherenceModel(model = lda_multicore_3,
                                  texts = clean_lemmas_leads,
                                  dictionary = dictionary,
                                  coherence = 'c_v'
                                 )
coh_score_multicore_3 = coh_model.get_coherence()
print(coh_score_multicore_3)

0.25041229421243233


Итого, наивысший coherence score для LDA Multicore достигается прои 4-х топиках

# Итоги

Изначально у меня было предположение, что могут выделиться в разные топики разные жанры - напр., фэнтези, научная фантастика, киберпанк и т.д., с характерными словами вроде "драконы", "магия", "космос" и подобными. К сожалению, предположение не оправдалось, чему есть несколько возможных объянений:
1. Коротких (в среднем 3-5 предложений) описаний книг недостаточно
2. При более тщательном анализе и очистке датасета могут получиться лучшие результаты
3. Так как по большей части эти тексты являются книжными рецензиями, они слишком похожи между собой

На практике модели выделяли топики скорее не по жанрам, а по типам статей.

In [147]:
coh_scores= {
    'LSI 3 topics' :[coh_score_lsi_3], 
    'LSI 4 topics': [coh_score_lsi_4], 
    'LSI 5 topics': [coh_score_lsi_5],
    'LDA 3 topics': [coh_score_lda_3], 
    'LDA 4 topics': [coh_score_lda_4], 
    'LDA 5 topics': [coh_score_lda_5],
    'LDA Multicore 3 topics': [coh_score_multicore_3], 
    'LDA Multicore 4 topics': [coh_score_multicore_4], 
    'LDA Multicore 5 topics': [coh_score_multicore_5]
}


In [151]:
coh_scores_df = pd.DataFrame.from_dict(coh_scores, orient = 'index', columns = ['Score'])

In [152]:
coh_scores_df

,Score
LSI 3 topics,0.364006
LSI 4 topics,0.778013
LSI 5 topics,0.479451
LDA 3 topics,0.320625
LDA 4 topics,0.317719
LDA 5 topics,0.261535
LDA Multicore 3 topics,0.250412
LDA Multicore 4 topics,0.290491
LDA Multicore 5 topics,0.272892


Как видно из таблицы, модель LSI показывает самый высокий coherence score. Наивысшего значения он достигает при числе топиков = 4. Действительно, по словам, выделенным этой моделью, можно успешно разделить тексты на отдельные, непересекающиеся топики, схожие с теми, что есть на сайте: книжные рекомендации, отрывки из произведений, короткие рассказы и, отдельной категорией, прочие книжные статьи.